Tutorial on Computing HFOs
==========================

In this tutorial, we will walk through how to compute HFOs on a sample dataset that is defined in [1]. 

We will demonstrate usage of the following detectors:

- Line Length detector
- RMS detector
- Morphology detector (used in the paper)

Dataset Preprocessing
---------------------
Note that the data has been converted to BIDS to facilitate easy loading using ``mne-bids`` package. Another thing to note is that the authors in this dataset 
reported HFOs detected using bipolar montage. In addition, they only analyzed HFOs for a subset of the recording channels.

In order to compare results to a monopolar reference, we define an HFO to be "found" if there was an HFO in either of the corresponding bipolar contacts.

References
----------
[1] Fedele T, Burnos S, Boran E, Krayenbühl N, Hilfiker P, Grunwald T, Sarnthein J. Resection of high frequency oscillations predicts seizure outcome in the individual patient.
Scientific Reports. 2017;7(1):13836.
https://www.nature.com/articles/s41598-017-13064-1
doi:10.1038/s41598-017-13064-1

In [1]:
# first let's load in all our packages
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import pandas as pd

from mne_bids import (read_raw_bids, BIDSPath, 
                      get_entity_vals, get_datatypes, 
                      make_report)
from mne_bids.stats import count_events

import mne
from mne import make_ad_hoc_cov
from mne.datasets import sample

basepath = os.path.join(os.getcwd(), "../..")
sys.path.append(basepath)
from mne_hfo import LineLengthDetector, RMSDetector
from mne_hfo.simulate import simulate_hfo, simulate_line_noise
from mne_hfo.utils import match_detections, _check_column_types, _change_duration_to_offset

# 1 Simulated Data

## 1.1 Simulate the HFO data

We will first create a simulated version of an HFO, which we can run through the detection algorithms to show a simple use case. We will create two HFOs, one in the ripple frequency (80-250 Hz) and one in the fast ripple frequency (250-500 Hz). We will then plot the simulated data

### 1.1.1 Create and plot the simulated HFOs

In [ ]:
# Simulate ripple and fast ripple HFOs
ripple_data, ripple_time = simulate_hfo(fs=2000, freq=120)
#ripple_data = ripple_data.reshape(1, len(ripple_data))
fast_ripple_data, fast_ripple_time = simulate_hfo(fs=2000, freq=400)

In [ ]:
fig = plt.figure()
ax = plt.axes()
ax.plot(ripple_time, ripple_data)
plt.title("Simulated Ripple")

fig = plt.figure()
ax = plt.axes()
ax.plot(fast_ripple_time, fast_ripple_data)
plt.title("Simulated Fast Ripple")

### 1.1.2 Add other "simulated" EEG data
These detectors work by finding patterns that deviate from the normal, so we need some "normal" EEG attached to the simulated HFO signal for the detectors to work. We can do this by grabbing a small segment of real EEG data from mne's samples.

In [ ]:
%%capture
# Load in template data
data_path = sample.data_path()
raw_fname = data_path + '/MEG/sample/sample_audvis_raw.fif'
fwd_fname = data_path + '/MEG/sample/sample_audvis-meg-eeg-oct-6-fwd.fif'

# Load real data as the template
raw = mne.io.read_raw_fif(raw_fname)
raw.set_eeg_reference(projection=True)
raw = raw.pick_channels(['EEG 001'])
sfreq = int(np.round(raw.info['sfreq']))
# Get two seconds of data to add before and after the HFO
raw_data_front = raw.get_data(start=0, stop=2*sfreq)[0]
raw_data_back = raw.get_data(start=2*sfreq, stop=4*sfreq)[0]
times = np.linspace(0, 2, 2*sfreq)

In [ ]:
# Visualize the raw data
fig = plt.figure()
ax = plt.axes()
ax.plot(times, raw_data_front)
plt.title("Raw Data Sample (Front)")

fig = plt.figure()
ax = plt.axes()
ax.plot(times, raw_data_back)
plt.title("Raw Data Sample (Back)")
#plt.plot(times, raw_data_back)

### 1.1.3 Combine the simulated data with sample data
Append the data to the front and back of the simulated HFOs

In [ ]:
def change_scale(val, aim_min, aim_max, cur_min, cur_max):
    return (val - cur_min) * (aim_max - aim_min) / (cur_max - cur_min) + aim_min

In [ ]:
# Normalize ripple data scale to that of the sample data
sig_min = np.min([np.min(raw_data_front), np.min(raw_data_back)])
sig_max = np.max([np.max(raw_data_front), np.max(raw_data_back)])
ripple_min = np.min(ripple_data)
ripple_max = np.max(ripple_data)
fast_ripple_min = np.min(fast_ripple_data)
fast_ripple_max = np.max(fast_ripple_data)

ripple_data = [change_scale(r, sig_min, sig_max, ripple_min, ripple_max) for r in ripple_data]
fast_ripple_data = [change_scale(r, sig_min, sig_max, fast_ripple_min, fast_ripple_max) for r in fast_ripple_data]

In [ ]:
# Combine the datasets and create new time vectors
ripple_data_total = np.concatenate((raw_data_front, ripple_data, raw_data_back))
t_total = len(raw_data_front) + len(ripple_data) + len(raw_data_back)
ripple_times_total = np.linspace(0, t_total/sfreq, t_total)

fast_ripple_data_total = np.concatenate((raw_data_front, fast_ripple_data, raw_data_back))
fast_t_total = len(raw_data_front) + len(fast_ripple_data) + len(raw_data_back)
fast_ripple_times_total = np.linspace(0, fast_t_total/sfreq, fast_t_total)


In [ ]:
# Visualize
fig = plt.figure()
ax = plt.axes()
ax.plot(ripple_times_total, ripple_data_total)
plt.title("Simulated data with Ripple")

fig = plt.figure()
ax = plt.axes()
ax.plot(fast_ripple_times_total, fast_ripple_data_total)
plt.title("Simulated data with Fast Ripple")

In [ ]:
# Reshape the data to fit detector requirements
ripple_data_total = ripple_data_total.reshape(1, len(ripple_data_total))
fast_ripple_data_total = fast_ripple_data_total.reshape(1, len(fast_ripple_data_total))

## 1.2 Detect the HFOs

We have two detectors that can detect the presense of an HFO, a LineLengthDetector and an RMSDetector. We will plug the simulated data into each detector to demonstrate the presence of HFOs.

### 1.2.1 Line Length Detector

In [ ]:
# Set Key Word Arguments for the Line Length Detector and generate the class object
kwargs = {
    'filter_band': (80, 250), # (l_freq, h_freq)
    'threshold': 3, # Number of st. deviations
    'win_size': 100, # Sliding window size in samples
    'overlap': 0.25, # Fraction of window overlap [0, 1]
    'sfreq': 2000,  # Sampling frequency of data
}
ll_detector = LineLengthDetector(**kwargs)

In [ ]:
# Detect HFOs in the raw data using the LineLengthDetector method.
# Return the class object with HFOs added
ll_detector = ll_detector.fit(ripple_data_total)

# Dictionary where keys are channel index and values are a list of tuples in the form of (start_samp, end_samp)
chs_hfo_dict = ll_detector.chs_hfos_ 
# nCh x nWin ndarray where each value is the line-length of the data window per channel
hfo_event_array = ll_detector.hfo_event_arr_
# Pandas dataframe containing onset, duration, sample trial, and trial type per HFO
hfo_df = ll_detector.df

### 1.2.2 Check Results
We should see a single HFO starting around 2 seconds and ending a small amount after

In [ ]:
detected_hfos = chs_hfo_dict.get("0")
for hfo in detected_hfos:
    start_samp = hfo[0]
    end_samp = hfo[1]
    start_time = start_samp / sfreq
    end_time = end_samp / sfreq
    print(f"Detected a ripple HFO starting at {start_time}s and ending at {end_time}s")

### 1.2.3 Repeat for RMS Detector
We should get almost the exact same result

In [ ]:
# Set Key Word Arguments for the Line Length Detector and generate the class object
kwargs = {
    'filter_band': (80, 250),
    'threshold': 3,
    'win_size': 100,
    'overlap': 0.25,
    'sfreq': 2000,
}
rms_detector = RMSDetector(**kwargs)
rms_detector = rms_detector.fit(ripple_data_total)
chs_hfo_dict = rms_detector.chs_hfos_ 

detected_hfos = chs_hfo_dict.get("0")
for hfo in detected_hfos:
    start_samp = hfo[0]
    end_samp = hfo[1]
    start_time = start_samp / sfreq
    end_time = end_samp / sfreq
    print(f"Detected a ripple HFO starting at {start_time}s and ending at {end_time}s")

### 1.2.4 Repeat for the fast ripple dataset
We should expect similar results

In [ ]:
# Line length Detector
kwargs = {
    'filter_band': (250, 500),
    'threshold': 3,
    'win_size': 100,
    'overlap': 0.25,
    'sfreq': 2000,
}
ll_detector = LineLengthDetector(**kwargs)
ll_detector = ll_detector.fit(fast_ripple_data_total)
chs_hfo_dict = ll_detector.chs_hfos_ 

detected_hfos = chs_hfo_dict.get("0")
for hfo in detected_hfos:
    start_samp = hfo[0]
    end_samp = hfo[1]
    start_time = start_samp / sfreq
    end_time = end_samp / sfreq
    print(f"Detected a fast ripple HFO starting at {start_time}s and ending at {end_time}s using the LineLength Detector")
    
# RMS detector
rms_detector = RMSDetector(**kwargs)
rms_detector = rms_detector.fit(fast_ripple_data_total)
chs_hfo_dict = rms_detector.chs_hfos_ 

detected_hfos = chs_hfo_dict.get("0")
for hfo in detected_hfos:
    start_samp = hfo[0]
    end_samp = hfo[1]
    start_time = start_samp / sfreq
    end_time = end_samp / sfreq
    print(f"Detected a fast ripple HFO starting at {start_time}s and ending at {end_time}s using the RMS detector")

### 1.2.5 More complex Example
Now let's combine the two datasets to show how the detectors differentiate the ripple types.

The detectors set to the ripple frequency band (80-250Hz) should only detect a single HFO (ripple) around the 2 second mark. The detectors set to the fast ripple frequency band (250-500Hz) should only detect a single HFO (fast ripple) around the 6 second mark (will vary based on the length of the ripple wave added).

In [ ]:
# Combine
hfo_data_total = np.concatenate((ripple_data_total, fast_ripple_data_total), axis=1)
hfo_times_total = np.linspace(0, hfo_data_total.shape[1]/sfreq, hfo_data_total.shape[1])

# Reshape just for plotting
hfo_data_total_plot = hfo_data_total.reshape(hfo_data_total.shape[1], 1)

# Plot
fig = plt.figure()
ax = plt.axes()
ax.plot(hfo_times_total, hfo_data_total_plot)
plt.title("Simulated data with Ripple and Fast Ripple")

In [ ]:
# First look for ripples
# Line length Detector
kwargs = {
    'filter_band': (80, 250),
    'threshold': 3,
    'win_size': 100,
    'overlap': 0.25,
    'sfreq': 2000,
}
ll_detector = LineLengthDetector(**kwargs)
ll_detector = ll_detector.fit(hfo_data_total)
chs_hfo_dict = ll_detector.chs_hfos_ 

detected_hfos = chs_hfo_dict.get("0")
for hfo in detected_hfos:
    start_samp = hfo[0]
    end_samp = hfo[1]
    start_time = start_samp / sfreq
    end_time = end_samp / sfreq
    print(f"Detected a ripple HFO starting at {start_time}s and ending at {end_time}s using the LineLength Detector")
    
# RMS detector
rms_detector = RMSDetector(**kwargs)
rms_detector = rms_detector.fit(hfo_data_total)
chs_hfo_dict = rms_detector.chs_hfos_ 

detected_hfos = chs_hfo_dict.get("0")
for hfo in detected_hfos:
    start_samp = hfo[0]
    end_samp = hfo[1]
    start_time = start_samp / sfreq
    end_time = end_samp / sfreq
    print(f"Detected a ripple HFO starting at {start_time}s and ending at {end_time}s using the RMS detector")

# Then look for fast ripples
# Line length Detector
kwargs = {
    'filter_band': (250, 500),
    'threshold': 3,
    'win_size': 100,
    'overlap': 0.25,
    'sfreq': 2000,
}
ll_detector = LineLengthDetector(**kwargs)
ll_detector = ll_detector.fit(hfo_data_total)
chs_hfo_dict = ll_detector.chs_hfos_ 

detected_hfos = chs_hfo_dict.get("0")
for hfo in detected_hfos:
    start_samp = hfo[0]
    end_samp = hfo[1]
    start_time = start_samp / sfreq
    end_time = end_samp / sfreq
    print(f"Detected a fast ripple HFO starting at {start_time}s and ending at {end_time}s using the LineLength Detector")
    
# RMS detector
rms_detector = RMSDetector(**kwargs)
rms_detector = rms_detector.fit(hfo_data_total)
chs_hfo_dict = rms_detector.chs_hfos_ 

detected_hfos = chs_hfo_dict.get("0")
for hfo in detected_hfos:
    start_samp = hfo[0]
    end_samp = hfo[1]
    start_time = start_samp / sfreq
    end_time = end_samp / sfreq
    print(f"Detected a fast ripple HFO starting at {start_time}s and ending at {end_time}s using the RMS detector")

# 2 Working with Real Data
We are now going to work with the dataset from Fedele et al. linked above

## 2.1 Load in Real Data

### 2.1.1 Define dataset paths and load the data

The data is assumed to be in BIDS format. We have converted the dataset into BIDS, 
which you can load using [mne-bids](https://github.com/mne-tools/mne-bids).

In [2]:
# this may change depending on where you store the data
root = '/Users/patrick/Dropbox/fedele_hfo_data'

In [3]:
# print a boiler plate summary report using mne-bids
report = make_report(root, verbose=False)
print(report)

Summarizing participants.tsv /Users/patrick/Dropbox/fedele_hfo_data/participants.tsv...
The iEEG Interictal Asleep HFO Dataset dataset was created with BIDS version
1.4.0 by Fedele T, Burnos S, Boran E, Krayenbühl N, Hilfiker P, Grunwald T, and
Sarnthein J.. This report was generated with MNE-BIDS
(https://doi.org/10.21105/joss.01896). The dataset consists of 20 participants
(comprised of 13 men and 6 women; handedness were all unknown; ages ranged from
17.0 to 52.0 (mean = 32.47, std = 11.43; 1 with unknown age))and 1 recording
sessions: interictalsleep. Data was recorded using a iEEG system (Neuralynx
manufacturer) sampled at 2000.0 Hz with line noise at 50.0 Hz using Sampling
with parameters 2000 Downsampled (Hz). There were 385 scans in total. Recording
durations ranged from 204.0 to 720.0 seconds (mean = 302.44, std = 37.85), for a
total of 116438.81 seconds of data recorded over all scans. For each dataset,
there were on average 51.88 (std = 15.55) recording channels per scan, ou

In [4]:
# there are iEEG datatypes throughout the dataset
datatypes = get_datatypes(root)
print(datatypes)

# get all the subjects, sessions
subjects = get_entity_vals(root, 'subject')
sessions = get_entity_vals(root, 'session')

print(subjects)
print(sessions)

['ieeg']
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']
['interictalsleep']


In [6]:
# construct BIDSPath for dataset we will demo
subjectID = subjects[0]
sessionID = sessions[0]
bids_path = BIDSPath(subject=subjectID, session=sessionID,
                     datatype='ieeg', 
                     suffix='ieeg',
                     extension='.vhdr', root=root)

# get all matching datasets
fpaths = bids_path.match()

print(f'Found {len(fpaths)} matching filepaths for this subject and session.')

Found 28 matching filepaths for this subject and session.


In [8]:
# analyze the first run
dataset_path = fpaths[0]
print(dataset_path)

# count the different HFO types (fr, ripple, frandr) that were detected using 
# the published Morphology detector
counts_df = count_events(dataset_path)
display(counts_df)

/Users/patrick/Dropbox/fedele_hfo_data/sub-01/ses-interictalsleep/ieeg/sub-01_ses-interictalsleep_run-01_ieeg.vhdr


NaN                                        \
trial_type                  fr_AHR1-2 fr_AHR2-3 fr_AHR3-4 fr_AL1-2 fr_AL2-3   
subject session         run                                                   
01      interictalsleep 01         15        23        12       15       19   

                                                                          ...  \
trial_type                  fr_AL3-4 fr_AR1-2 fr_AR2-3 fr_AR3-4 fr_HL1-2  ...   
subject session         run                                               ...   
01      interictalsleep 01        20       25       28       11        9  ...   

                                                                       \
trial_type                  ripple_IAR2-3 ripple_IAR3-4 ripple_IAR4-5   
subject session         run                                             
01      interictalsleep 01            351           223           179   

                                                                       \
trial_type                  ripple_IAR5-6 ripple_IPR1-2 ripple_IPR2-3   
subject session         run                                             
01      interictalsleep 01            238            12            32   

                                                                       \
trial_type                  ripple_IPR3-4 ripple_PHR1-2 ripple_PHR2-3   
subject session         run                                             
01      interictalsleep 01            128           121           111   

                                           
trial_type                  ripple_PHR3-4  
subject session         run                
01      interictalsleep 01             64  

[1 rows x 60 columns]

In [9]:
%%capture
# load dataset into mne Raw object
raw = read_raw_bids(dataset_path)

In [ ]:
# Optional - Change backend to interactive for below plot
# Allows you to scroll dataset, highlight (or grey out) certain channels, zoom in/out, etc.
%matplotlib notebook

In [ ]:
# Plot the raw data with vertical colorbars to denote where HFOs were detected
raw_plot = raw.plot()
raw_plot.show()
print('plotting channels with HFO events detected in '
      'the original publication in color.')

In [ ]:
# Optional - Change back to regular plots
%matplotlib inline

In [ ]:
print(f'Data has sampling rate of {raw.info["sfreq"]} Hz')

### 2.1.2 Load Annotated HFOs

In [10]:
# All annotated HFO events for this file
annotations = raw.annotations
annot = annotations[0]
[print(key, end="\t") for key in annot.keys()]
print("\n")
for annot in annotations:
    [print(val, end="\t") for val in annot.values()]
    print("\n")

onset	duration	description	orig_time	

0.0015	0.016	fr_PHR1-2	2013-12-07 08:29:20+00:00	

0.0015	0.0515	ripple_AHR3-4	2013-12-07 08:29:20+00:00	

0.0525	0.0695	ripple_HL2-3	2013-12-07 08:29:20+00:00	

0.053	0.051	ripple_HL3-4	2013-12-07 08:29:20+00:00	

0.1795	0.042	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

0.268	0.084	ripple_AR1-2	2013-12-07 08:29:20+00:00	

0.2875	0.064	ripple_AR2-3	2013-12-07 08:29:20+00:00	

0.3705	0.0425	ripple_IAR4-5	2013-12-07 08:29:20+00:00	

0.398	0.053	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

0.4085	0.0355	ripple_AR2-3	2013-12-07 08:29:20+00:00	

0.4975	0.035	ripple_HL1-2	2013-12-07 08:29:20+00:00	

0.8725	0.0105	fr_IAR1-2	2013-12-07 08:29:20+00:00	

0.919	0.052	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

0.9355	0.035	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

1.0895	0.065	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

1.244	0.0405	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

1.812	0.0915	ripple_HL3-4	2013-12-07 08:29:20+00:00	

1.9795	0.0555	ripple_IPR3-4	2013-12-07 0


25.639	0.0115	fr_IAR5-6	2013-12-07 08:29:20+00:00	

25.667	0.0375	ripple_IAR5-6	2013-12-07 08:29:20+00:00	

25.681	0.106	ripple_IAR3-4	2013-12-07 08:29:20+00:00	

25.7685	0.06	ripple_IAR5-6	2013-12-07 08:29:20+00:00	

25.7995	0.061	ripple_HL1-2	2013-12-07 08:29:20+00:00	

25.8015	0.04	ripple_HL3-4	2013-12-07 08:29:20+00:00	

25.861	0.0475	ripple_PHR1-2	2013-12-07 08:29:20+00:00	

25.9245	0.012	fr_IPR1-2	2013-12-07 08:29:20+00:00	

25.94	0.0775	ripple_AR1-2	2013-12-07 08:29:20+00:00	

25.99	0.052	ripple_AHR1-2	2013-12-07 08:29:20+00:00	

25.991	0.0395	ripple_AHR2-3	2013-12-07 08:29:20+00:00	

26.434	0.0455	ripple_HL3-4	2013-12-07 08:29:20+00:00	

26.54	0.0475	ripple_HL2-3	2013-12-07 08:29:20+00:00	

26.5995	0.018	fr_IPR2-3	2013-12-07 08:29:20+00:00	

26.7175	0.0365	ripple_IPR3-4	2013-12-07 08:29:20+00:00	

26.924	0.034	ripple_HL3-4	2013-12-07 08:29:20+00:00	

27.06	0.053	ripple_AL1-2	2013-12-07 08:29:20+00:00	

27.099	0.066	ripple_AR1-2	2013-12-07 08:29:20+00:00	

27.1	0.068	ripple_AR2

49.086	0.0365	ripple_HL1-2	2013-12-07 08:29:20+00:00	

49.2675	0.014	fr_AHR2-3	2013-12-07 08:29:20+00:00	

49.474	0.06	ripple_IAR4-5	2013-12-07 08:29:20+00:00	

49.5925	0.1555	ripple_HL2-3	2013-12-07 08:29:20+00:00	

49.6175	0.0615	ripple_HL3-4	2013-12-07 08:29:20+00:00	

49.658	0.048	ripple_HL1-2	2013-12-07 08:29:20+00:00	

49.705	0.037	ripple_HL3-4	2013-12-07 08:29:20+00:00	

49.9215	0.035	ripple_IPR2-3	2013-12-07 08:29:20+00:00	

50.042	0.036	ripple_HL2-3	2013-12-07 08:29:20+00:00	

50.1715	0.0555	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

50.429	0.065	ripple_IAR5-6	2013-12-07 08:29:20+00:00	

50.4765	0.0865	ripple_IPR3-4	2013-12-07 08:29:20+00:00	

50.6375	0.0425	ripple_HL3-4	2013-12-07 08:29:20+00:00	

50.661	0.049	ripple_AHR2-3	2013-12-07 08:29:20+00:00	

50.665	0.0455	ripple_AHR1-2	2013-12-07 08:29:20+00:00	

50.666	0.0365	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

50.6695	0.0405	ripple_AHR3-4	2013-12-07 08:29:20+00:00	

50.6745	0.064	ripple_PHR2-3	2013-12-07 08:29:20+00:00	

50.6

69.095	0.0495	ripple_HL3-4	2013-12-07 08:29:20+00:00	

69.1185	0.048	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

69.267	0.0275	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

69.316	0.068	ripple_HL3-4	2013-12-07 08:29:20+00:00	

69.3855	0.0445	ripple_HL1-2	2013-12-07 08:29:20+00:00	

69.541	0.05	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

69.5505	0.042	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

69.895	0.052	ripple_HL3-4	2013-12-07 08:29:20+00:00	

69.908	0.038	ripple_HL2-3	2013-12-07 08:29:20+00:00	

70.011	0.0675	ripple_HL3-4	2013-12-07 08:29:20+00:00	

70.0155	0.067	ripple_HL2-3	2013-12-07 08:29:20+00:00	

70.144	0.0415	ripple_HL3-4	2013-12-07 08:29:20+00:00	

70.148	0.043	ripple_IPR3-4	2013-12-07 08:29:20+00:00	

70.3055	0.0385	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

70.484	0.1335	ripple_HL1-2	2013-12-07 08:29:20+00:00	

70.5385	0.076	ripple_IAR4-5	2013-12-07 08:29:20+00:00	

70.5385	0.104	ripple_HL3-4	2013-12-07 08:29:20+00:00	

70.8655	0.067	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

70.983

87.8405	0.154	ripple_AHR2-3	2013-12-07 08:29:20+00:00	

87.8675	0.0975	ripple_AR2-3	2013-12-07 08:29:20+00:00	

87.8765	0.1455	ripple_AHR3-4	2013-12-07 08:29:20+00:00	

87.8845	0.106	ripple_AR1-2	2013-12-07 08:29:20+00:00	

87.8845	0.106	fr_AR1-2	2013-12-07 08:29:20+00:00	

87.8845	0.106	frandr_AR1-2	2013-12-07 08:29:20+00:00	

87.8955	0.0235	fr_PHR1-2	2013-12-07 08:29:20+00:00	

87.904	0.0625	ripple_AR3-4	2013-12-07 08:29:20+00:00	

87.9055	0.086	ripple_AHR1-2	2013-12-07 08:29:20+00:00	

87.907	0.0675	ripple_PHR3-4	2013-12-07 08:29:20+00:00	

87.907	0.0905	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

87.923	0.0595	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

88.005	0.0735	ripple_IAR3-4	2013-12-07 08:29:20+00:00	

88.0105	0.08	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

88.1085	0.064	ripple_HL3-4	2013-12-07 08:29:20+00:00	

88.411	0.041	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

88.567	0.051	ripple_HL1-2	2013-12-07 08:29:20+00:00	

88.682	0.0635	ripple_HL2-3	2013-12-07 08:29:20+00:00	

88.706	0

...

161.579	0.0615	ripple_HL3-4	2013-12-07 08:29:20+00:00	

161.8015	0.056	ripple_AL1-2	2013-12-07 08:29:20+00:00	

161.8055	0.045	ripple_HL2-3	2013-12-07 08:29:20+00:00	

161.8055	0.0675	ripple_HL3-4	2013-12-07 08:29:20+00:00	

161.994	0.111	ripple_AR2-3	2013-12-07 08:29:20+00:00	

161.994	0.111	fr_AR2-3	2013-12-07 08:29:20+00:00	

161.994	0.111	frandr_AR2-3	2013-12-07 08:29:20+00:00	

161.9965	0.0935	ripple_AR1-2	2013-12-07 08:29:20+00:00	

162.0235	0.0525	ripple_AHR1-2	2013-12-07 08:29:20+00:00	

162.0355	0.0315	ripple_AHR3-4	2013-12-07 08:29:20+00:00	

162.269	0.0615	ripple_HL3-4	2013-12-07 08:29:20+00:00	

162.4265	0.118	ripple_AR1-2	2013-12-07 08:29:20+00:00	

162.4285	0.033	ripple_IPR3-4	2013-12-07 08:29:20+00:00	

162.444	0.0535	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

162.485	0.0885	ripple_HL3-4	2013-12-07 08:29:20+00:00	

162.509	0.0615	ripple_HL1-2	2013-12-07 08:29:20+00:00	

162.526	0.052	ripple_PHR1-2	2013-12-07 08:29:20+00:00	

162.6385	0.0475	ripple_IAR1-2	2013-12-07 08:29:

191.6545	0.0425	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

191.744	0.0515	ripple_AR1-2	2013-12-07 08:29:20+00:00	

191.7455	0.0485	ripple_AR2-3	2013-12-07 08:29:20+00:00	

191.7785	0.0235	fr_AL1-2	2013-12-07 08:29:20+00:00	

191.7945	0.055	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

191.823	0.057	ripple_AR1-2	2013-12-07 08:29:20+00:00	

191.836	0.0425	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

191.8365	0.0425	ripple_AR2-3	2013-12-07 08:29:20+00:00	

192.021	0.0505	ripple_AR2-3	2013-12-07 08:29:20+00:00	

192.044	0.1245	ripple_HL3-4	2013-12-07 08:29:20+00:00	

192.224	0.1095	ripple_AR2-3	2013-12-07 08:29:20+00:00	

192.225	0.118	ripple_AR1-2	2013-12-07 08:29:20+00:00	

192.3255	0.0445	ripple_PHR1-2	2013-12-07 08:29:20+00:00	

192.3255	0.0445	fr_PHR1-2	2013-12-07 08:29:20+00:00	

192.3255	0.0445	frandr_PHR1-2	2013-12-07 08:29:20+00:00	

192.435	0.047	ripple_IAR4-5	2013-12-07 08:29:20+00:00	

192.4355	0.0515	ripple_IAR5-6	2013-12-07 08:29:20+00:00	

192.7815	0.1715	ripple_HL1-2	2013-12-07 08:

218.6385	0.0345	ripple_HL1-2	2013-12-07 08:29:20+00:00	

218.677	0.0115	fr_IAR4-5	2013-12-07 08:29:20+00:00	

218.6795	0.016	fr_IAR5-6	2013-12-07 08:29:20+00:00	

218.7095	0.0855	ripple_AL1-2	2013-12-07 08:29:20+00:00	

218.749	0.0725	ripple_HL2-3	2013-12-07 08:29:20+00:00	

218.793	0.045	ripple_HL1-2	2013-12-07 08:29:20+00:00	

218.822	0.1325	ripple_HL3-4	2013-12-07 08:29:20+00:00	

218.8435	0.1135	ripple_HL2-3	2013-12-07 08:29:20+00:00	

218.8825	0.052	ripple_IAR3-4	2013-12-07 08:29:20+00:00	

219.0275	0.0575	ripple_IAR3-4	2013-12-07 08:29:20+00:00	

219.0275	0.0615	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

219.1525	0.0385	ripple_HL3-4	2013-12-07 08:29:20+00:00	

219.153	0.039	ripple_HL2-3	2013-12-07 08:29:20+00:00	

219.3655	0.046	ripple_AR1-2	2013-12-07 08:29:20+00:00	

219.3655	0.053	ripple_AR2-3	2013-12-07 08:29:20+00:00	

219.382	0.024	ripple_HL2-3	2013-12-07 08:29:20+00:00	

219.425	0.012	fr_IAR4-5	2013-12-07 08:29:20+00:00	

219.629	0.0395	ripple_IAR5-6	2013-12-07 08:29:20+00:

254.975	0.094	ripple_AR2-3	2013-12-07 08:29:20+00:00	

254.9765	0.088	ripple_AR1-2	2013-12-07 08:29:20+00:00	

255.0065	0.09	ripple_IAR3-4	2013-12-07 08:29:20+00:00	

255.037	0.0675	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

255.262	0.082	ripple_HL1-2	2013-12-07 08:29:20+00:00	

255.3465	0.049	ripple_HL3-4	2013-12-07 08:29:20+00:00	

255.614	0.1695	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

255.6185	0.0325	ripple_HL2-3	2013-12-07 08:29:20+00:00	

255.62	0.2735	ripple_IAR3-4	2013-12-07 08:29:20+00:00	

255.639	0.095	ripple_IAR5-6	2013-12-07 08:29:20+00:00	

255.691	0.178	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

255.84	0.051	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

255.9265	0.04	ripple_IAR3-4	2013-12-07 08:29:20+00:00	

255.9285	0.0405	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

255.9435	0.0345	ripple_HL2-3	2013-12-07 08:29:20+00:00	

256.022	0.08	ripple_HL2-3	2013-12-07 08:29:20+00:00	

256.1365	0.049499999999970894	ripple_HL1-2	2013-12-07 08:29:20+00:00	

256.542	0.011500000000029103	fr_


279.506	0.042	ripple_HL3-4	2013-12-07 08:29:20+00:00	

279.663	0.032	ripple_IAR3-4	2013-12-07 08:29:20+00:00	

279.663	0.034	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

279.7425	0.0565	ripple_IAR4-5	2013-12-07 08:29:20+00:00	

279.7705	0.1015	ripple_IAR1-2	2013-12-07 08:29:20+00:00	

279.9325	0.012	fr_AR3-4	2013-12-07 08:29:20+00:00	

279.95	0.0635	ripple_AR1-2	2013-12-07 08:29:20+00:00	

279.9585	0.0555	ripple_AR2-3	2013-12-07 08:29:20+00:00	

280.4155	0.0385	ripple_HL3-4	2013-12-07 08:29:20+00:00	

280.473	0.0105	fr_AL2-3	2013-12-07 08:29:20+00:00	

280.6015	0.0135	fr_IAR4-5	2013-12-07 08:29:20+00:00	

280.609	0.0525	ripple_HL1-2	2013-12-07 08:29:20+00:00	

280.618	0.059	ripple_HL3-4	2013-12-07 08:29:20+00:00	

280.7155	0.0385	ripple_IAR3-4	2013-12-07 08:29:20+00:00	

280.7915	0.0125	fr_IPR3-4	2013-12-07 08:29:20+00:00	

280.8175	0.034	ripple_IAR2-3	2013-12-07 08:29:20+00:00	

280.877	0.011	fr_AL1-2	2013-12-07 08:29:20+00:00	

280.908	0.0355	ripple_IAR3-4	2013-12-07 08:29:20+00:00	

2

In [11]:
# Convert to convenient data structure (pandas DF)
column_names = ["onset", "duration", "sample", "trial_type"]

sfreq = raw.info["sfreq"]
rows = []
for annot in annotations:
    onset = float(annot.get("onset"))
    duration = float(annot.get("duration"))
    sample = onset * sfreq
    trial_type = annot.get("description")
    annot_row = [onset, duration, sample, trial_type]
    rows.append(annot_row)

gs_df = pd.concat([pd.DataFrame([row], columns=column_names) for row in rows],
          ignore_index=True)
print(gs_df)

         onset  duration    sample     trial_type
0       0.0015    0.0160       3.0      fr_PHR1-2
1       0.0015    0.0515       3.0  ripple_AHR3-4
2       0.0525    0.0695     105.0   ripple_HL2-3
3       0.0530    0.0510     106.0   ripple_HL3-4
4       0.1795    0.0420     359.0  ripple_IAR1-2
...        ...       ...       ...            ...
4218  299.6765    0.0845  599353.0   ripple_HL1-2
4219  299.6945    0.0675  599389.0   ripple_HL2-3
4220  299.7685    0.0815  599537.0  ripple_IAR5-6
4221  299.8490    0.0340  599698.0  ripple_PHR2-3
4222  300.0000    0.0000  600000.0   ripple_HL3-4

[4223 rows x 4 columns]


In [41]:
# for now, lets just look at ripple events:
gs_df_ripple = gs_df[gs_df['trial_type'].str.contains("ripple")]
new_indices = range(1, gs_df_ripple.shape[0]+1)
gs_df_ripple = gs_df_ripple.reindex(new_indices)
print(gs_df_ripple)

         onset  duration    sample     trial_type    offset
1       0.0015    0.0515       3.0  ripple_AHR3-4    0.0530
2       0.0525    0.0695     105.0   ripple_HL2-3    0.1220
3       0.0530    0.0510     106.0   ripple_HL3-4    0.1040
4       0.1795    0.0420     359.0  ripple_IAR1-2    0.2215
5       0.2680    0.0840     536.0   ripple_AR1-2    0.3520
...        ...       ...       ...            ...       ...
3640  263.9745    0.1245  527949.0  ripple_PHR1-2  264.0990
3641       NaN       NaN       NaN            NaN       NaN
3642       NaN       NaN       NaN            NaN       NaN
3643  263.9870    0.0615  527974.0  ripple_IAR2-3  264.0485
3644       NaN       NaN       NaN            NaN       NaN

[3644 rows x 5 columns]


## 2.2 Detect HFOs


### 2.2.1 Line Length Detector

In [12]:
# Set Key Word Arguments for the Line Length Detector and generate the class object
kwargs = {
    'filter_band': (80, 250), # (l_freq, h_freq)
    'threshold': 3, # Number of st. deviations
    'win_size': 100, # Sliding window size in samples
    'overlap': 0.25, # Fraction of window overlap [0, 1]
    'hfo_name': "ripple"
}
ll_detector = LineLengthDetector(**kwargs)

In [13]:
# Detect HFOs in the raw data using the LineLengthDetector method.
# Return the class object with HFOs added
ll_detector = ll_detector.fit(raw)

# Dictionary where keys are channel index and values are a list of tuples in the form of (start_samp, end_samp)
ll_chs_hfo_dict = ll_detector.chs_hfos_ 
# nCh x nWin ndarray where each value is the line-length of the data window per channel
ll_hfo_event_array = ll_detector.hfo_event_arr_
# Pandas dataframe containing onset, duration, sample trial, and trial type per HFO
ll_hfo_df = ll_detector.df

100%|██████████| 50/50 [00:13<00:00,  3.72it/s]


In [ ]:
print(ll_hfo_df.sort_values(by=['onset']))

### 2.2.2 RMS Detector


In [14]:
# Set Key Word Arguments for the RMS Detector and generate the class object
kwargs = {
    'filter_band': (80, 250),
    'threshold': 3,
    'win_size': 100,
    'overlap': 0.25,
    'hfo_name': 'ripple',
}
rms_detector = RMSDetector(**kwargs)

In [15]:
# Detect HFOs in the raw data using the RMSDetector method.
rms_detector = rms_detector.fit(raw)

rms_chs_hfo_dict = rms_detector.chs_hfos_ 
rms_hfo_event_array = rms_detector.hfo_event_arr_
rms_hfo_df = rms_detector.df

100%|██████████| 50/50 [00:07<00:00,  6.37it/s]


In [ ]:
print(rms_hfo_df.sort_values(by=['onset']))

## 2.3 Compare Results

### 2.3.1 Normalize the Dataframes
The Fedele paper reports HFO events in channel pairs. We are assuming that an event with trial_type ripple_HL2-3 had a ripple at this window at BOTH HL2 and HL3. Therefore, the output dataframes from the LineLength Detector and RMS Detector need to be modified to meet this criteria. 

In [16]:
def _check_neighbor_row(row, df):
    """
    Return whether a neighbor channel shares an onset time.
    
    row and df need to contain the same column names, specifically trial_type
    TODO: We probably want some tolerance here, if the onsets are ever so slightly off
    """
    row_trial = row.trial_type
    trial_type = row_trial.split("_")[0]
    row_channel = row_trial.split("_")[1]
    channel_number = int("".join([s for s in row_channel if s.isdigit()]))
    channel_name = ''.join([i for i in row_channel if not i.isdigit()])
    neighbor_trial = f"{trial_type}_{channel_name}{channel_number+1}"
    neighbor_row = df.loc[df["trial_type"] == neighbor_trial]
    combined_trial = f"{row_trial}-{channel_number+1}"
    if neighbor_row.empty:
        return False
    return combined_trial

In [18]:
column_names = ["onset", "duration", "sample", "trial_type"]
ll_hfo_array = []
ll_onsets = ll_hfo_df.onset.unique()
for onset_val in ll_onsets:
    onset_df = ll_hfo_df.loc[ll_hfo_df['onset'] == onset_val]
    for row_group in onset_df.iterrows():
        row = row_group[1]
        new_trial = _check_neighbor_row(row, onset_df)
        if new_trial:
            row.trial_type = new_trial
            ll_hfo_array.append(row.values.tolist())
ll_hfo_df_mod = pd.concat([pd.DataFrame([row], columns=column_names) for row in ll_hfo_array],
          ignore_index=True)
print(ll_hfo_df_mod.sort_values(by=['onset']))

rms_hfo_array = []
rms_onsets = rms_hfo_df.onset.unique()
for onset_val in ll_onsets:
    onset_df = rms_hfo_df.loc[rms_hfo_df['onset'] == onset_val]
    for row_group in onset_df.iterrows():
        row = row_group[1]
        new_trial = _check_neighbor_row(row, onset_df)
        if new_trial:
            row.trial_type = new_trial
            rms_hfo_array.append(row.values.tolist())
rms_hfo_df_mod = pd.concat([pd.DataFrame([row], columns=column_names) for row in rms_hfo_array],
          ignore_index=True)
print(rms_hfo_df_mod.sort_values(by=['onset']))


       onset duration  sample     trial_type
143      0.0   0.0625       0   ripple_HL1-2
142      0.0   0.0625       0   ripple_AL3-4
141      0.0   0.0625       0  ripple_AHR6-7
140      0.0   0.0625       0  ripple_AHR5-6
139      0.0   0.0625       0  ripple_AHR4-5
..       ...      ...     ...            ...
16   95.8375   0.0625  191675   ripple_AL4-5
17   95.8375   0.0625  191675   ripple_AR6-7
18   95.8375   0.0625  191675   ripple_HL7-8
387  96.5375   0.0625  193075   ripple_AL6-7
430     96.6   0.0625  193200   ripple_HL1-2

[507 rows x 4 columns]
    onset             duration  sample     trial_type
132   0.0               0.0625       0  ripple_AHR4-5
136   0.0               0.0625       0   ripple_HL3-4
133   0.0               0.0625       0  ripple_AHR5-6
134   0.0               0.0625       0  ripple_AHR6-7
135   0.0               0.0625       0   ripple_HL2-3
..    ...                  ...     ...            ...
573  96.6  0.07500000000000284  193200   ripple_HL7-8
568 

In [33]:
# Make sure all column types are numeric
ll_hfo_df_mod, rms_hfo_df_mod =  _check_column_types([ll_hfo_df_mod, rms_hfo_df_mod], ["onset", "duration", "sample"])
# Add an offset column to each df
gs_df_ripple, ll_hfo_df_mod, rms_hfo_df_mod = _change_duration_to_offset([gs_df_ripple, ll_hfo_df_mod, rms_hfo_df_mod], ["onset", "duration"])
print(gs_df)

         onset  duration    sample     trial_type    offset
0       0.0015    0.0160       3.0      fr_PHR1-2    0.0175
1       0.0015    0.0515       3.0  ripple_AHR3-4    0.0530
2       0.0525    0.0695     105.0   ripple_HL2-3    0.1220
3       0.0530    0.0510     106.0   ripple_HL3-4    0.1040
4       0.1795    0.0420     359.0  ripple_IAR1-2    0.2215
...        ...       ...       ...            ...       ...
4218  299.6765    0.0845  599353.0   ripple_HL1-2  299.7610
4219  299.6945    0.0675  599389.0   ripple_HL2-3  299.7620
4220  299.7685    0.0815  599537.0  ripple_IAR5-6  299.8500
4221  299.8490    0.0340  599698.0  ripple_PHR2-3  299.8830
4222  300.0000    0.0000  600000.0   ripple_HL3-4  300.0000

[4223 rows x 5 columns]


/Users/patrick/Desktop/mne-hfo/mne_hfo/utils.py:264: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['offset'] = df.iloc[:, df_col_indices].sum(axis=1)


### 2.3.2 Find matches
Now that our dataframes are in the same format, we can compare them. We will do that by iterating over trial types (i.e. combo of hfo types and channel name) on the "gold standard" dataset and see if the two detectors have matching 

In [42]:
column_names = ["onset", "duration", "sample", "trial_type"]
gs_ll_match = []
gs_rms_match = []

unique_trials = gs_df_ripple.trial_type.unique()

for trial in unique_trials:
    gs_trial_df = gs_df_ripple.loc[gs_df_ripple['trial_type'] == trial]
    ll_trial_df = ll_hfo_df_mod.loc[ll_hfo_df_mod['trial_type'] == trial]
    rms_trial_df = rms_hfo_df_mod.loc[rms_hfo_df_mod['trial_type'] == trial]
    gs_ll_match.append(match_detections(gs_trial_df, ll_trial_df, ['onset', 'offset'], sec_unit=1.0))
    gs_rms_match.append(match_detections(gs_trial_df, rms_trial_df, ['onset', 'offset'], sec_unit=1.0))

gs_ll_match_df = pd.concat([row for row in gs_ll_match], ignore_index=True).dropna()
print(gs_ll_match_df)
gs_rms_match_df = pd.concat([row for row in gs_rms_match], ignore_index=True).dropna()
print(gs_rms_match_df)

/usr/local/Caskroom/miniconda/base/envs/mne-hfo-conda/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/Caskroom/miniconda/base/envs/mne-hfo-conda/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/Caskroom/miniconda/base/envs/mne-hfo-conda/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be se

/usr/local/Caskroom/miniconda/base/envs/mne-hfo-conda/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/Caskroom/miniconda/base/envs/mne-hfo-conda/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/Caskroom/miniconda/base/envs/mne-hfo-conda/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be se

     gs_index dd_index
2        88.0    226.0
8       208.0    332.0
10      367.0    237.0
22     1085.0    308.0
26     1283.0    335.0
...       ...      ...
3114   3110.0    339.0
3116   3209.0    463.0
3119   3406.0    278.0
3127   3596.0    284.0
3131      296      193

[144 rows x 2 columns]
     gs_index dd_index
1        43.0    247.0
7       181.0    520.0
22     1085.0    351.0
28     1485.0    282.0
29     1515.0    360.0
...       ...      ...
3103   2764.0    627.0
3104   2794.0    628.0
3114   3110.0    403.0
3116   3209.0    632.0
3124   3550.0    321.0

[158 rows x 2 columns]


/usr/local/Caskroom/miniconda/base/envs/mne-hfo-conda/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/Caskroom/miniconda/base/envs/mne-hfo-conda/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/Caskroom/miniconda/base/envs/mne-hfo-conda/lib/python3.8/site-packages/pandas/core/frame.py:3188: SettingWithCopyWarning: 
A value is trying to be se

In [43]:
for ind, row in gs_ll_match_df.iterrows():
    gs_row = gs_df_ripple.iloc[[int(row['gs_index'])]]
    ll_row = ll_hfo_df_mod.iloc[[int(row['dd_index'])]]
    print(gs_row, ll_row)
    

    onset  duration  sample trial_type  offset
89    NaN       NaN     NaN        NaN     NaN       onset  duration  sample     trial_type  offset
226  6.4625    0.0875   12925  ripple_AHR3-4    6.55
       onset  duration   sample     trial_type   offset
209  14.3045     0.114  28609.0  ripple_AHR1-2  14.4185      onset  duration  sample     trial_type  offset
332   14.3       0.1   28600  ripple_AHR3-4    14.4
       onset  duration   sample    trial_type   offset
368  24.1925     0.035  48385.0  ripple_HL1-2  24.2275        onset  duration  sample     trial_type  offset
237  24.2125    0.0875   48425  ripple_AHR3-4    24.3
        onset  duration    sample     trial_type   offset
1086  74.8245     0.045  149649.0  ripple_PHR3-4  74.8695       onset  duration  sample     trial_type  offset
308  74.825     0.075  149650  ripple_AHR3-4    74.9
        onset  duration    sample    trial_type   offset
1284  87.8845     0.106  175769.0  ripple_AR1-2  87.9905        onset  duration  sample

365  24.185     0.086  48370.0  ripple_AHR2-3  24.271        onset  duration  sample     trial_type   offset
238  24.2125       0.1   48425  ripple_PHR2-3  24.3125
      onset  duration   sample     trial_type   offset
478  32.747    0.0555  65494.0  ripple_IAR5-6  32.8025      onset  duration  sample     trial_type   offset
242   32.8    0.0875   65600  ripple_PHR2-3  32.8875
      onset  duration    sample     trial_type   offset
824  59.891    0.0675  119782.0  ripple_AHR2-3  59.9585        onset  duration  sample     trial_type   offset
304  59.9125       0.1  119825  ripple_PHR2-3  60.0125
      onset  duration  sample trial_type  offset
1160    NaN       NaN     NaN        NaN     NaN        onset  duration  sample     trial_type  offset
311  80.2375    0.0625  160475  ripple_PHR2-3    80.3
        onset  duration    sample     trial_type    offset
1509  106.053    0.1165  212106.0  ripple_AHR2-3  106.1695         onset  duration  sample     trial_type    offset
315  106.0875    

253  143.925    0.0875  287850  ripple_PHR1-2  144.0125
      onset  duration  sample trial_type  offset
2101    NaN       NaN     NaN        NaN     NaN         onset  duration  sample     trial_type  offset
168  146.3625    0.0875  292725  ripple_PHR1-2  146.45
      onset  duration  sample trial_type  offset
2483    NaN       NaN     NaN        NaN     NaN         onset  duration  sample     trial_type    offset
268  177.3875     0.075  354775  ripple_PHR1-2  177.4625
      onset  duration  sample trial_type  offset
2502    NaN       NaN     NaN        NaN     NaN       onset  duration  sample     trial_type   offset
457  177.85     0.075  355700  ripple_PHR1-2  177.925
      onset  duration  sample trial_type  offset
2714    NaN       NaN     NaN        NaN     NaN         onset  duration  sample     trial_type    offset
402  194.1375     0.075  388275  ripple_PHR1-2  194.2125
      onset  duration  sample trial_type  offset
2765    NaN       NaN     NaN        NaN     NaN         

In [ ]:
gs_hfo_num = gs_df.shape[0]

In [ ]:
def calculate_events_per_min(raw, events):
    sfreq = raw.info['sfreq']
    total_samps = raw.n_times
    total_minutes = total_samps / (sfreq * 60.0)
    return len(events) / total_minutes